In [8]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy

from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering


In [9]:
# Load models
nlp = spacy.load("en_core_web_sm")
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")


In [10]:
files = {
    "PG": '1094591345_us.csv',
    "Giant": '1536111825_us.csv',
    "Trek": '1609966547_us.csv',
    "Shimano": '1437969979_us.csv',
    "Ninebot US": '1484302191_us.csv'
}


In [11]:
def auto_label_cluster(texts, top_k=3):
    noun_phrases = []
    keywords = []

    for text in texts:
        doc = nlp(text)

        # 1️noun phrases
        for chunk in doc.noun_chunks:
            phrase = chunk.text.lower().strip()
            if len(phrase.split()) >= 2:
                noun_phrases.append(phrase)

        # 2️fallback keywords
        for token in doc:
            if token.pos_ in ["NOUN", "PROPN", "ADJ"] and not token.is_stop:
                if len(token.text) > 2:
                    keywords.append(token.text.lower())

    # Priority 1: noun phrases
    if noun_phrases:
        return ", ".join([p for p, _ in Counter(noun_phrases).most_common(top_k)])

    # Priority 2: keywords
    if keywords:
        return ", ".join([w for w, _ in Counter(keywords).most_common(top_k)])

    # Priority 3: fallback label
    return "other language / short feedback"



In [12]:
def run_review_clustering(df, n_clusters=6):
    texts = df["content"].astype(str).tolist()

    embeddings = embedder.encode(texts, show_progress_bar=False)

    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        metric="cosine",
        linkage="average"
    )

    labels = clustering.fit_predict(embeddings)
    df = df.copy()
    df["cluster"] = labels

    return df


In [13]:
results = {}

for app, path in files.items():
    df = pd.read_csv(path)

    for group_name, sentiments in {
        "neg_neu": ["negative", "neutral"],
        "pos": ["positive"]
    }.items():

        subset = df[df["sentiment"].isin(sentiments)]

        if len(subset) < 20:
            continue  # too small to cluster

        clustered = run_review_clustering(subset, n_clusters=6)

        cluster_labels = {}

        for cid in sorted(clustered.cluster.unique()):
            texts = clustered[clustered.cluster == cid]["content"].tolist()
            label = auto_label_cluster(texts)
            cluster_labels[cid] = label

        results[(app, group_name)] = {
            "data": clustered,
            "labels": cluster_labels
        }


In [ ]:
for (app, group), result in results.items():
    print(f"\n===== {app} | {group.upper()} =====")

    for cid, label in result["labels"].items():
        print(f"\nCluster {cid} → {label}")

        samples = (
            result["data"]
            .query("cluster == @cid")["content"]
            .head(3)
            .tolist()
        )

        for s in samples:
            print("-", s[:120])


===== PG | NEG_NEU =====

Cluster 0 → the game, this game, the app
- I like the game….but it’s just so frustrating you move so slowly and I just wish that you could just click to where you 
- WE NEED MORE RAID TIME ITS SO STUPID THAT THERES A TIME LIMIT FOR RAIDS ADD MORE TIME AND WHY IS THERE A DAILY GOLD LIMI
- Lately, my game has gotten very buggy. Every couple minutes, the game will kick me out and I end up losing chances to fi

Cluster 1 → the game, my character, a special research
- Just wondering why I was given a special research that needed to be done within a certain time frame. I thought those ar
- Not to be that guy, but every week my character gets reset to a very light pigment, with blonde hair and blue eyes like 
- I can’t move a that YouTube person mystic 7 has the weirdest office people ever and guess what I’m mystic 7

Cluster 2 → возвращайтесь, россию, 是因为我是在大陆吗
- возвращайтесь в россию
- 是因为我是在大陆吗
- Gotta go everywhere

Cluster 3 → the chinese
- It’s the worst
- Th